In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

#import models
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

from sklearn import model_selection
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from tabulate import _table_formats, tabulate
from sklearn.preprocessing import StandardScaler

In [2]:
datas = '/Users/pkrok/OneDrive/Python/Giannakopoulos/Data.xlsx'

In [3]:
pd.set_option("display.max_columns", None)
df=pd.read_excel(datas)
df.head(2)
df['logD'] = np.log10(df['diffusivity'])
# df = df.dropna()
df

,type,cell,aperture,aperture2,ExpansionRatio,M-N,Μ-N_lff,Μ-N_kFF,ionicRad,MetalCharge,apertureAtom_charge,apertureAtom_σ,apertureAtom_e,linker_length1,linker_length2,linker_length3,linker_mass1,linker_mass1.1,linker_mass3,func1_length,func2_length,func3_length,func1_mass,func2_mass,func3_mass,func1_charge,func2_charge,func3_charge,gas,mass,Dbulk,ascentricF,size - van der Waals (Å),size - kinetic diameter (Å),diffusivity,solubility,permeability,Unnamed: 37,Unnamed: 38,Unnamed: 39,aperture2.1,Dresults,number of modifications,logD
0,CdIF-1,1.8077,3.92,3.93000,1.002551,2.18,2.257,45354.60,92,1.1901,NaN,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001,He,4.002,0.000121,-0.390,2.66,2.60,6.000000e-08,NaN,NaN,NaN,NaN,NaN,596.0,522.0,72.0,-7.221849
1,CdIF-1,1.8077,3.92,NaN,0.000000,2.18,2.257,45354.60,92,1.1901,NaN,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001,H2,2.010,0.000108,-0.217,3.13,2.89,1.700000e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.769551
2,CdIF-1,1.8077,3.92,NaN,0.000000,2.18,2.257,45354.60,92,1.1901,NaN,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001,O2,31.999,0.000015,0.022,2.94,3.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CdIF-1,1.8077,3.92,3.97000,1.012755,2.18,2.257,45354.60,92,1.1901,NaN,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001,CO2,44.010,0.000009,0.225,3.24,3.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CdIF-1,1.8077,3.92,NaN,0.000000,2.18,2.257,45354.60,92,1.1901,NaN,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001,N2,28.000,0.000015,0.037,3.13,3.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,dIm_Mg,NaN,2.95,3.08837,1.046905,NaN,2.139,38994.88,71,1.9871,-0.185,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710,ethane,30.070,0.000007,0.099,3.72,4.00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
849,dIm_Mg,NaN,2.95,3.12791,1.060308,NaN,2.139,38994.88,71,1.9871,-0.185,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710,propylene,42.080,0.000005,0.142,4.03,4.50,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
850,dIm_Mg,NaN,2.95,3.17050,1.074746,NaN,2.139,38994.88,71,1.9871,-0.185,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710,propane,44.100,0.000005,0.152,4.16,4.30,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
851,dIm_Mg,NaN,2.95,NaN,0.000000,NaN,2.139,38994.88,71,1.9871,-0.185,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710,butane,58.120,0.000003,0.200,4.52,4.50,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Keep only the columns I need

In [4]:
df=df[[ 'aperture', 'size - van der Waals (Å)','logD']]
df.head()

,aperture,size - van der Waals (Å),logD
0,3.92,2.66,-7.221849
1,3.92,3.13,-8.769551
2,3.92,2.94,NaN
3,3.92,3.24,NaN
4,3.92,3.13,NaN


In [5]:
df=df.rename(columns={'size - van der Waals (Å)':'diameter' })

In [6]:
df = df.dropna()
df=df.reset_index(drop=True)
df.head()

,aperture,diameter,logD
0,3.92,2.66,-7.221849
1,3.92,3.13,-8.769551
2,3.92,3.25,-8.585027
3,3.92,3.59,-9.236572
4,3.92,3.72,-9.197226


# Create my dataset x and y

In [7]:
x = np.asanyarray(df[['aperture', 'diameter']])
y = np.asanyarray(df[['logD']])

x1 = (df[['aperture', 'diameter']])
y1 = (df[['logD']])

# Scale the Data

In [8]:
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)

# Create GridSeachCV for NN regressor

In [15]:
# parameters= {'hidden_layer_sizes': [n for n in range(10,600)],
#      'solver': ['lbfgs', 'sgd', 'adam'],
#      'max_iter': [n for n in range(100,8000)]}

parameters= {'hidden_layer_sizes': [n for n in range(100,600, 200)],
     'solver': ['adam','lbfgs', 'sgd' ],
     'max_iter': [n for n in range(1000, 6000, 200)]}

NNregr = MLPRegressor(random_state=1)

In [16]:
nn_cv = GridSearchCV(NNregr, parameters)
nn_cv.fit(x, y.ravel())

C:\Users\pkrok\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\pkrok\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\pkrok\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITE

GridSearchCV(estimator=MLPRegressor(random_state=1),
             param_grid={'hidden_layer_sizes': [100, 300, 500],
                         'max_iter': [1000, 1200, 1400, 1600, 1800, 2000, 2200,
                                      2400, 2600, 2800, 3000, 3200, 3400, 3600,
                                      3800, 4000, 4200, 4400, 4600, 4800, 5000,
                                      5200, 5400, 5600, 5800],
                         'solver': ['adam', 'lbfgs', 'sgd']})

In [18]:
print("tuned hpyerparameters :(best parameters) ", nn_cv.best_params_)
print("accuracy :",nn_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'hidden_layer_sizes': 300, 'max_iter': 1000, 'solver': 'lbfgs'}
accuracy : 0.7463157970422812
